In [9]:
def greedy_algorithm(n: int):
    """
    Greedy algorithm for TTP data.
    Input:
        - n: int --> number of participating teams (even number)
    Output:
        - initial_sol: dict --> initial solution for double round-robin, where keys are rounds and values are lists of matchups
    """
    
    # Initialize solution dictionary
    initial_sol = dict()
    for round_num in range(n - 1):
        initial_sol[round_num] = []
    
    # Create list of lexopgraphical matchups    
    lexographical_matchups = [(i, j) for i in range(n - 1) for j in range(i + 1, n)]
    
    # Fill solution dictionary with matchups
    matchup_clusters_added = 0
    while matchup_clusters_added < n/2:
        new_matchups_added = 0
        round_num = 0
        while new_matchups_added < n - 1:
            if initial_sol[round_num] == []:    # Check if round is empty and add matchup
                initial_sol[round_num].append(lexographical_matchups[0])
                lexographical_matchups.pop(0)
                new_matchups_added += 1
            else:   # Check for team conflicts in round
                visited_nums = []
                for round_match in initial_sol[round_num]:
                    visited_nums.append(round_match[0])
                    visited_nums.append(round_match[1])
                if lexographical_matchups[0][0] not in visited_nums and lexographical_matchups[0][1] not in visited_nums:   # Condition for adding matchup if round isn't empty
                    initial_sol[round_num].append(lexographical_matchups[0])
                    lexographical_matchups.pop(0)
                    new_matchups_added += 1
                visited_nums = []
            round_num += 1
            round_num %= n - 1  # Reset round number after reaching last round
        matchup_clusters_added += 1
    
    # Add mirrored results to complete the double round-robin
    for round_num in range(n - 1):
        initial_sol[n - 1 + round_num] = []
        for matchup in initial_sol[round_num]:
            initial_sol[n - 1 + round_num].append((matchup[1], matchup[0]))
    
    return initial_sol 

In [10]:
import gurobipy as gp
from gurobipy import GRB

# --- 1. Parameters and Sets ---
# Define the number of teams (n) and the maximum trip length (L)
N = 4
U = 3 #upperbound
L = 2 #lowerbound 
DAYS = 2 * N - 2  # Total number of days

# Sets for teams and days
Teams = range(1, N + 1)
Days_X = range(1, DAYS + 1)      # Days k for x and z 
Days_Y = range(1, DAYS)          # Days k for y (1 to 5, as it involves k+1)
Days_U = range(1, DAYS - U + 1)  # Days k for constraints (4) and (5) (1 to 2n-2-L)
Days_L = range(1, DAYS - L + 1)  # Days k for constraints (10) and (11) (1 to 2n-2-L)
################################################################################################
# THIS MUST BE REPLACED WITH THE ACTUAL DISTANCED
D = {
    (1, 1): 0, (1, 2): 10, (1, 3): 20, (1, 4): 30,
    (2, 1): 10, (2, 2): 0, (2, 3): 5, (2, 4): 15,
    (3, 1): 20, (3, 2): 5, (3, 3): 0, (3, 4): 25,
    (4, 1): 30, (4, 2): 15, (4, 3): 25, (4, 4): 0
}
###############################################################################################
# 2. Sample Input Schedule (x_tilde) from Local Search #### MUST BE REPLACED BY LOCAL SEARCH FUNCTION

INPUT_X_SCHEDULE = {
    # Day 1 (k=0): 1 hosts 2 (0, 1), 3 hosts 4 (2, 3)
    (1, 2, 1): 1, (3, 4, 1): 1,
    # Day 2 (k=1): 1 hosts 3 (0, 2), 2 hosts 4 (1, 3)
    (1, 3, 2): 1, (2, 4, 2): 1,
    # Day 3 (k=2): 1 hosts 4 (0, 3), 2 hosts 3 (1, 2)
    (1, 4, 3): 1, (2, 3, 3): 1,
    # Day 4 (k=3): 2 hosts 1 (1, 0), 4 hosts 3 (3, 2)
    (2, 1, 4): 1, (4, 3, 4): 1,
    # Day 5 (k=4): 3 hosts 1 (2, 0), 4 hosts 2 (3, 1)
    (3, 1, 5): 1, (4, 2, 5): 1,
    # Day 6 (k=5): 4 hosts 1 (3, 0), 3 hosts 2 (2, 1)
    (4, 1, 6): 1, (3, 2, 6): 1,
}
# Fill in zeros for non-games
for i in Teams:
    for j in Teams:
        for k in Days_X:
            if (i, j, k) not in INPUT_X_SCHEDULE:
                INPUT_X_SCHEDULE[(i, j, k)] = 0


def build_base_model(distances, N, L, U, DAYS, Teams, Days_X, Days_Y, Days_U, Days_L):
    """
    Constructs the base TTP subproblem model including variables, objective,
    and the core TTP constraints (1) through (9).
    """
    model = gp.Model("TTP_Subproblem")
    model.setParam('OutputFlag', 0) # makes the output cleaner
    model.setParam('TimeLimit', 30.0)
    # --- 3. Decision Variables ---
    # x[i, j, k]: 1 if team i hosts team j on day k (Binary)
    x = model.addVars(Teams, Teams, Days_X, vtype=GRB.BINARY, name="x")

    # z[i, j, k]: Auxiliary variable (Continuous)
    z = model.addVars(Teams, Teams, Days_X, vtype=GRB.BINARY, name="z")

    # y[t, i, j, k]: Travel variable for team t from location i to j on day k (Continuous, LB=0)
    # k is the index of the FIRST day in the transition (k to k+1)
    y = model.addVars(Teams, Teams, Teams, Days_Y, vtype=GRB.BINARY, lb=0, name="y")

    # --- 4. Objective Function ---
    # min sum(i,j) d_ij * x_ij1 + sum(t,i,j) sum(k=1 to 2n-3) d_ij * y_tijk + sum(i,j) d_ij * x_ij,2n-2
    obj = gp.quicksum(distances[i, j] * x[i, j, 1] for i in Teams for j in Teams) + \
          gp.quicksum(distances[i, j] * y[t, i, j, k]
                      for t in Teams for i in Teams for j in Teams for k in Days_Y) + \
          gp.quicksum(distances[i, j] * x[i, j, DAYS] for i in Teams for j in Teams)

    model.setObjective(obj, GRB.MINIMIZE)

    # --- 5. Constraints (TTP Core) ---

    # (1) No self-play
    model.addConstrs((x[i, i, k] == 0 for i in Teams for k in Days_X), name="No_Self_Play")

    # (2) Play one game per day 
    model.addConstrs((gp.quicksum(x[i, j, k] + x[j, i, k] for j in Teams) == 1
                      for i in Teams for k in Days_X), name="Play_One_Game_Per_Day")

    # (3) Play each team twice 
    model.addConstrs((gp.quicksum(x[i, j, k] for k in Days_X) == 1
                      for i in Teams for j in Teams if i != j), name="Play_Each_Team_Twice")

    # (4) Max home stay
    model.addConstrs((gp.quicksum(x[i, j, k + u] for j in Teams for u in range(U + 1)) <= U
                      for i in Teams for k in Days_U), name="Max_Home_Stay")

    # (5) Max road trip
    model.addConstrs((gp.quicksum(x[i, j, k + u] for i in Teams for u in range(U + 1)) <= U
                      for j in Teams for k in Days_U), name="Max_Road_Trip")
    
    # (6) No game repeats 2 times back to back
    model.addConstrs((x[i, j, k] + x[j, i, k] + x[i, j, k + 1] + x[j, i, k + 1] <= 1
                      for i in Teams for j in Teams for k in Days_Y if i != j),
                      name="No_Back_to_Back_Game")

    # --- 6. Driving behavior of the teams constraints ---

    # (7)
    model.addConstrs((z[i, i, k] == gp.quicksum(x[i, j, k] for j in Teams)
                      for i in Teams for k in Days_X), name="Def_z_home")

    # (8))
    model.addConstrs((z[i, j, k] == x[i, j, k]
                      for i in Teams for j in Teams for k in Days_X if i != j), name="Def_z_game")

    # (9)
    model.addConstrs((y[t, i, j, k] >= z[t, i, k] + z[t, j, k + 1] - 1
                      for t in Teams for i in Teams for j in Teams for k in Days_Y),
                     name="Def_y_travel_link")
    # (10)  Min home stay (Lower Bound L): Forces team i to be home at least L times over L+1 days.
    #model.addConstrs((gp.quicksum(x[i, j, k + l] for j in Teams for l in range(L + 1)) >= L
    #                  for i in Teams for k in Days_L), name="Min_Home_Stay_L")

    # (11) Min road trip (Lower Bound L): Forces team j to be away at least L times over L+1 days.
    #model.addConstrs((gp.quicksum(x[i, j, k + l] for i in Teams for l in range(L + 1)) >= L
    #                  for j in Teams for k in Days_L), name="Min_Road_Trip_L")

    return model, x, z, y


def solve_ttp_subproblem(mode: str, distances: dict, input_schedule: dict):
    """
    Solves the TTP subproblem IP given an input schedule from local search,
    specialized by the optimization mode ('HA-Opt' or 'Non-HA-Opt').

    Args:
        mode (str): 'HA-Opt' or 'Non-HA-Opt'.
        distances (dict): The distance matrix D[i, j].
        input_schedule (dict): The initial solution x_tilde[(i, j, k)].
    """
    if mode not in ['HA-Opt', 'Non-HA-Opt']:
        print("Error: Mode must be 'HA-Opt' or 'Non-HA-Opt'.")
        return None

    # 1. Build the base model (variables, objective, and core constraints 1-9)
    model, x, z, y = build_base_model(distances, N, L, U, DAYS, Teams, Days_X, Days_Y, Days_U, Days_L)

    # 2. get the input schedule
    for i in Teams:
        for j in Teams:
            for k in Days_X:
                x[i, j, k].Start = input_schedule[(i, j, k)]

    # 3. Add Subproblem-Specific Constraints (Constraints 12, 13, or 14 from the paper)
    if mode == 'HA-Opt':
        print("HA-Optimization Mode: Fixing Matchups, Optimizing Venues (Constraint 12)")
        for i in Teams:
            for j in Teams:
                if i < j:
                    for k in Days_X:
                        matchup_fixed_val = input_schedule[(i, j, k)] + input_schedule[(j, i, k)]
                        model.addConstr(x[i, j, k] + x[j, i, k] == matchup_fixed_val,
                                        name=f"FixMatchup_{i}_{j}_{k}")

    elif mode == 'Non-HA-Opt':
        print("Non-HA-Optimization Mode: Fixing Venues, Optimizing Matchups (Constraints 13 & 14)")
        # constraint 14
        for i in Teams:
            for k in Days_X:
                home_status_fixed = gp.quicksum(input_schedule[(i, j, k)] for j in Teams)
                model.addConstr(gp.quicksum(x[i, j, k] for j in Teams) == home_status_fixed,
                                name=f"FixHomeStatus_{i}_{k}")
        # constraint 13
        for j in Teams:
            for k in Days_X:
                # Calculate the required away status (0 or 1) from the input schedule
                away_status_fixed = gp.quicksum(input_schedule[(i, j, k)] for i in Teams)
                model.addConstr(gp.quicksum(x[i, j, k] for i in Teams) == away_status_fixed,
                                name=f"FixAwayStatus_{j}_{k}")

    # 4. Solve the model
    model.optimize()

    # 5. Output Results
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Optimization Status: {model.Status}")
        print(f"Objective Value: {model.objVal:,.0f}")
        
        # Return the new schedule x, z, y values
        return {
            'x': {v.varName: v.X for v in x.values() if v.X > 0.5},
            'z': {v.varName: v.X for v in z.values() if v.X > 0.5},
            'y': {v.varName: v.X for v in y.values() if v.X > 0.001},
            'objVal': model.objVal
        }
    else:
        print(f"Model could not be solved to optimality/time limit. Status: {model.Status}")
        return None

print(f"Starting TTP Subproblem for N={N}, L={L}, DAYS={DAYS}...")
print("This code demonstrates the two subproblems used for local search refinement.")
    
# DEMO 1: HA Optimization (Fixes who plays whom, optimizes where)
ha_result = solve_ttp_subproblem(
    mode='HA-Opt',
    distances=D,
    input_schedule=INPUT_X_SCHEDULE
)

# DEMO 2: Non-HA Optimization (Fixes home/away pattern, optimizes opponents)
non_ha_result = solve_ttp_subproblem(
    mode='Non-HA-Opt',
    distances=D,
    input_schedule=INPUT_X_SCHEDULE
)

if ha_result:
    print(f"HA-Opt Final Objective: {ha_result['objVal']:,.0f}")
if non_ha_result:
    print(f"Non-HA-Opt Final Objective: {non_ha_result['objVal']:,.0f}")


Starting TTP Subproblem for N=4, L=2, DAYS=6...
This code demonstrates the two subproblems used for local search refinement.
HA-Optimization Mode: Fixing Matchups, Optimizing Venues (Constraint 12)
Optimization Status: 2
Objective Value: 205
Non-HA-Optimization Mode: Fixing Venues, Optimizing Matchups (Constraints 13 & 14)
Optimization Status: 2
Objective Value: 395
HA-Opt Final Objective: 205
Non-HA-Opt Final Objective: 395
